In [ ]:
# !pip install datasets
# !pip install pandas
# !pip install pillow

In [1]:
from huggingface_hub import notebook_login
from datasets import load_dataset
from tqdm import tqdm

notebook_login()

In [8]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import logging


In [3]:
image_dataset = load_dataset("CognitiveLab/image-prompts-raw")

# image_dataset.save_to_disk("path/to/dataset/directory")


Generating train split:   0%|          | 0/12683 [00:00<?, ? examples/s]

In [7]:
image_dataset["train"]

Dataset({
    features: ['negativePrompt', 'steps', 'id', 'sampler', 'imageUrl', 'cfgScale', 'clipSkip', 'url', 'model', 'prompt', 'seed'],
    num_rows: 12683
})

In [5]:
import pandas as pd

pandas_data = pd.DataFrame(image_dataset["train"])

pandas_data.head()

,negativePrompt,steps,id,sampler,imageUrl,cfgScale,clipSkip,url,model,prompt,seed
0,"nsfw, paintings, sketches, (worst quality:2), ...",25,127497,DPM++ SDE Karras,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,12,2,https://civitai.com/images/127497,chilloutmix_NiPrunedFp32Fix,"best quality, (photorealistic:1.4), 1girl, (bl...",3118275117
1,"paintings, sketches, (worst quality:2), (low q...",30,295008,DPM++ SDE Karras,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,7,2,https://civitai.com/images/295008,chilloutmix_NiPrunedFp32Fix,"(RAW photo, best quality), (realistic, photo-r...",1684432449
2,"(worst quality, low quality, illustration, 3d,...",46,2008378,DPM++ 2M Karras,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,7,,https://civitai.com/images/2008378,sdvn6Realxl_detailface,Amazing detailed photography of a perfect beau...,2814388551
3,"(worst quality:1.4),(low quality:1.4), NSFW,cr...",22,3140516,DPM++ 2M Karras,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,8,,https://civitai.com/images/3140516,,"(masterpiece, best quality,hires,high resoluti...",1105145580557338
4,"(worst quality:1.4), (low quality:1.4), (monoc...",20,256477,DPM++ 2M Karras,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,7,2,https://civitai.com/images/256477,luckyStrikeMix_V02Realistic,"(RAW photo, best quality), (realistic, photo-r...",3589893419


In [6]:
image_url = "https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/7e1b47b4-cfe2-4b33-2c9d-6bc58031d500/width=768/7e1b47b4-cfe2-4b33-2c9d-6bc58031d500.jpeg"

response = requests.get(image_url)
response.raise_for_status()  # Check for errors in the HTTP request

# Open the image using PIL
image = Image.open(BytesIO(response.content))

# Save the image to the specified path
image.save(f'data/{1}.jpg', 'JPEG')

```json
[
  {
    "id": "{id}",
    "image": "data/{id}.jpg",
    "conversations": [
      {
        "from": "human",
        "value": "<image>\nWrite a prompt for Stable Diffusion to generate this image."
      },
      {
        "from": "gpt",
        "value": "{prompt}"
      },
    ]
  },
]
```

In [ ]:
# Set up logging
logging.basicConfig(filename='download_log.txt', level=logging.ERROR)

# Assuming your DataFrame is named 'pandas_data'
# Replace 'pandas_data' with your actual DataFrame name

for index, row in tqdm(pandas_data.iterrows(), total=len(pandas_data), desc="Downloading Images"):
    image_url = row['imageUrl']
    image_id = row['id']
    
    try:
        response = requests.get(image_url)
        response.raise_for_status()  # Check for errors in the HTTP request

        # Open the image using PIL
        image = Image.open(BytesIO(response.content))

        # Save the image to the specified path
        image.save(f'data/{image_id}.jpg', 'JPEG')
        # tqdm.write(f"Image {image_id} downloaded successfully.")
    except Exception as e:
        error_message = f"Error downloading image {image_id}: {str(e)}"
        tqdm.write(error_message)
        
        # Log the exception to the specified file
        logging.error(f"Image ID: {image_id}, URL: {image_url}\n{error_message}")

In [20]:
import pandas as pd
import json

# Assuming your DataFrame is named 'pandas_data'
# Replace 'pandas_data' with your actual DataFrame name

# Convert DataFrame to JSON
json_data = pandas_data.to_json(orient='records', lines=False)
data_list = json.loads(json_data)

# Create a new list to store the formatted data
formatted_data = []

for record in data_list:
    
    formatted_prompt = {
                "prompt" : record["prompt"],
                "negativePrompt" : record["negativePrompt"],
                "steps" : record["steps"],
                "sampler" : record["sampler"],
                "model" : record["model"],
                "cfgScale" : record["cfgScale"],
                "clipSkip" : record["clipSkip"],
                }
    
    formatted_record = {
        "id": record["id"],
        "image": f"data/{record['id']}.jpg",
        "conversations": [
            {"from": "human", "value": f"<image>\n Write a prompt for Stable Diffusion to generate this image in json formate :"},
            {"from": "gpt", "value": f"""{formatted_prompt}"""}
        ]
    }
    formatted_data.append(formatted_record)

# Save the formatted data to a JSON file
output_file_path = './data/formatted_data.json'
with open(output_file_path, 'w') as json_file:
    json.dump(formatted_data, json_file, indent=2)

print(f"Formatted data saved to {output_file_path}")


Formatted data saved to ./data/formatted_data.json


In [18]:
formatted_dataset = load_dataset("json", data_files="./formatted_data.json")

Generating train split: 0 examples [00:00, ? examples/s]